In [ ]:
!pip install torch torchvision --quiet
!pip install transformers  --quiet
!pip install pandas  --quiet
!pip install numpy  --quiet
!pip install sentencepiece  --quiet
!pip install sentence-splitter  --quiet
!pip install shap --quiet
!pip install optuna --quiet




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.9/547.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.3/225.3 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 6.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Imports**

In [ ]:
import random
import torch
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW,AutoModelForQuestionAnswering, get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
from transformers import DebertaTokenizer, DebertaModel, BartTokenizer
import math

# **Model loading**

In [ ]:
# Use a GPU if you have one available (Runtime -> Change runtime type -> GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set seeds for reproducibility
random.seed(26)
np.random.seed(26)
torch.manual_seed(26)

tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-large", do_lower_case=True)


In [ ]:
def get_optimizer_grouped_parameters(
    model, model_type,
    learning_rate, weight_decay,
    layerwise_learning_rate_decay
):
    no_decay = ["bias", "LayerNorm.weight"]
    # initialize lr for task specific layer
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if "classifier" in n or "pooler" in n],
            "weight_decay": 0.0,
            "lr": learning_rate,
        },
    ]
    # initialize lrs for every layer
    num_layers = model.config.num_hidden_layers
    layers = [getattr(model, model_type).embeddings] + list(getattr(model, model_type).encoder.layer)
    layers.reverse()
    lr = learning_rate
    for layer in layers:
        lr *= layerwise_learning_rate_decay
        optimizer_grouped_parameters += [
            {
                "params": [p for n, p in layer.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": weight_decay,
                "lr": lr,
            },
            {
                "params": [p for n, p in layer.named_parameters() if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
                "lr": lr,
            },
        ]
    return optimizer_grouped_parameters

In [ ]:
def encode_data(tokenizer, passages,questions, max_length):
    """Encode the question/passage pairs into features than can be fed to the model."""
    input_ids = []
    attention_masks = []

    for passage,question in zip(passages,questions):
        encoded_data = tokenizer.encode_plus(passage,question, max_length=max_length, pad_to_max_length=True, truncation='longest_first')
        encoded_pair = encoded_data["input_ids"]
        attention_mask = encoded_data["attention_mask"]

        input_ids.append(encoded_pair)
        attention_masks.append(attention_mask)

    return np.array(input_ids), np.array(attention_masks)

In [ ]:
def predict(passage,question):
  sequence = tokenizer.encode_plus(passage,question,max_length=512, pad_to_max_length=True, truncation='longest_first', return_tensors="pt")['input_ids'].to(device)

  logits = model(sequence)[0]
  probabilities = torch.softmax(logits, dim=1).detach().cpu().tolist()[0]
  proba_yes = round(probabilities[1], 2)
  proba_no = round(probabilities[0], 2)

  #print(f"Question: {question}, Yes: {proba_yes}, No: {proba_no}")

  if (proba_yes >= proba_no):
    return True
  else:
    return False






In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split



# Train and evaluate the accuracy of neural network with the addition of pruning mechanism
def train_and_evaluate(model,train_data_df,dev_data_df):
    passages_train = train_data_df.Abstract.values
    questions_train = train_data_df.questions.values
    answers_train = train_data_df.AMES.values.astype(int)

    passages_dev = dev_data_df.Abstract.values
    questions_dev = dev_data_df.questions.values
    answers_dev = dev_data_df.AMES.values.astype(int)

    # Encoding data
    max_seq_length = 512
    input_ids_train, attention_masks_train = encode_data(tokenizer, passages_train,questions_train, max_seq_length)
    input_ids_dev, attention_masks_dev = encode_data(tokenizer, passages_dev,questions_dev, max_seq_length)

    train_features = (input_ids_train, attention_masks_train, answers_train)
    dev_features = (input_ids_dev, attention_masks_dev, answers_dev)

    batch_size = 2
    train_features_tensors = [torch.tensor(feature, dtype=torch.long) for feature in train_features]
    dev_features_tensors = [torch.tensor(feature, dtype=torch.long) for feature in dev_features]

    train_dataset = TensorDataset(*train_features_tensors)
    dev_dataset = TensorDataset(*dev_features_tensors)

    train_sampler = RandomSampler(train_dataset)
    dev_sampler = SequentialSampler(dev_dataset)

    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)
    dev_dataloader = DataLoader(dev_dataset, sampler=dev_sampler, batch_size=batch_size)

    learning_rate = 5e-5
    layerwise_learning_rate_decay = 0.9
    adam_epsilon = 1e-6
    use_bertadam = False

    # scheduler params
    num_epochs = 5
    num_warmup_steps = 0

    optimizer = AdamW(model.parameters(),
        lr=learning_rate,
        eps=adam_epsilon,
        correct_bias=not use_bertadam
    )
    scheduler = get_cosine_schedule_with_warmup(
        optimizer,
        num_warmup_steps=num_warmup_steps,
        num_training_steps=num_epochs
    )

    print("Done setting up optimizer\n")
    train_loss_values = []
    dev_acc_values = []
    state=[]

    for i in tqdm(range(num_epochs), desc="Epoch"):

      # Training
      print("In epoch ", i, "\n")
      epoch_train_loss = 0 # Cumulative loss
      model.train()
      #model.zero_grad()

      for step, batch in enumerate(train_dataloader):

          input_ids = batch[0].to(device)
          attention_masks = batch[1].to(device)
          labels = batch[2].to(device)
          model.zero_grad()
          outputs = model(input_ids, token_type_ids=None, attention_mask=attention_masks, labels=labels)

          loss = outputs[0]
          #loss = loss / grad_acc_steps
          epoch_train_loss += loss.item()

          loss.backward()
          torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
          optimizer.step()
          scheduler.step()

      epoch_train_loss = epoch_train_loss / len(train_dataloader)
      train_loss_values.append(epoch_train_loss)
      print("Epoch loss is", epoch_train_loss)

      # Evaluation
      epoch_dev_accuracy = 0 # Cumulative accuracy
      model.eval()

      for batch in dev_dataloader:

        input_ids = batch[0].to(device)
        attention_masks = batch[1].to(device)
        labels = batch[2]

        with torch.no_grad():
            outputs = model(input_ids, token_type_ids=None, attention_mask=attention_masks)

        logits = outputs[0]
        logits = logits.detach().cpu().numpy()

        predictions = np.argmax(logits, axis=1).flatten()
        labels = labels.numpy().flatten()

        epoch_dev_accuracy += np.sum(predictions == labels) / len(labels)

      epoch_dev_accuracy = epoch_dev_accuracy / len(dev_dataloader)
      print("Epoch accuracy is",epoch_dev_accuracy )
      dev_acc_values.append(epoch_dev_accuracy)
      temp=model
      state.append(temp)

    index = dev_acc_values.index(max(dev_acc_values))
    best = state[index]
    return best



In [ ]:
import pandas as pd

df=pd.read_csv('./new_data_mutagenicity.csv')

# Apply the function to the column
df['label'] = df['label'].apply(lambda x: x.startswith("['True"))
#df['label'] = df['label'].apply(lambda x: x[0] == "['False - AMES non Mutagenic']")

new_column_names = {'text': 'Abstract', 'label': 'AMES'}
df.rename(columns=new_column_names, inplace=True)


In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
import pandas as pd
import gc
#df=pd.read_csv('./data.csv')

kf = KFold(n_splits=5, random_state=42, shuffle=True)
CV_accuracy_array=[]
CV_macro_avg_array=[]
CV_weighted_avg_array=[]
for train_index, test_index in kf.split(df):
    print("TRAIN:", train_index)
    print("TEST:", test_index)
    train_data_df, dev_data_df = df.loc[train_index], df.loc[test_index]
    model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-large")
    model.to(device)
    model=train_and_evaluate(model,train_data_df,dev_data_df)
    filter=list(dev_data_df.index)
    preds=[]
    index=[]
    for i in filter:
        try:
          passage=dev_data_df.Abstract[i]
          question =dev_data_df.questions[i]
          answer=predict(passage,question)
          preds.append(answer)
          index.append(i)
        except Exception as e:
          print(e)
          continue
    dev=dev_data_df[dev_data_df.index.isin(index)]
    true_results=dev['AMES'].tolist()
    print(classification_report(true_results, preds))
    results = classification_report(true_results, preds,output_dict=True,)
    CV_accuracy_array.append(results['accuracy'])
    CV_macro_avg_array.append(results['macro avg']['f1-score'])
    CV_weighted_avg_array.append(results['weighted avg']['f1-score'])
    del model
    gc.collect()
    torch.cuda.empty_cache()



TRAIN: [   0    1    2 ... 1643 1644 1645]
TEST: [  15   23   29   30   32   43   44   49   51   56   59   63   65   67
   69   70   73   76   78   99  100  101  107  109  115  123  124  128
  135  141  148  162  163  168  170  173  175  184  185  192  198  199
  203  212  220  226  231  237  239  240  244  247  251  259  261  266
  270  271  274  275  289  297  298  300  303  306  309  316  324  331
  332  339  342  344  350  351  352  353  366  367  371  374  383  394
  398  405  408  411  413  414  415  416  420  422  425  426  433  438
  450  451  464  471  479  481  482  483  486  490  493  494  497  526
  527  529  534  538  543  551  552  554  560  561  567  575  582  584
  585  588  589  590  591  597  610  613  614  617  619  620  621  629
  651  654  668  669  674  679  680  682  694  706  707  720  724  727
  730  741  744  752  754  764  765  767  772  780  792  798  802  803
  809  810  813  816  818  838  842  844  845  847  855  861  865  867
  873  874  879  887  888  8

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-large and are newly initialized: ['pooler.dense.weight', 'classifier.bias', 'pooler.dense.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2393: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Done setting up optimizer



/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

In epoch  0 

Epoch loss is 0.708608222930312
Epoch accuracy is 0.5272727272727272
In epoch  1 

Epoch loss is 0.6993126750263712
Epoch accuracy is 0.5272727272727272
In epoch  2 

Epoch loss is 0.712444632657562
Epoch accuracy is 0.5272727272727272
In epoch  3 

Epoch loss is 0.6951574403707825
Epoch accuracy is 0.5272727272727272
In epoch  4 

Epoch loss is 0.7095212600503417
Epoch accuracy is 0.5272727272727272


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

              precision    recall  f1-score   support

       False       0.00      0.00      0.00       156
        True       0.53      1.00      0.69       173

    accuracy                           0.53       329
   macro avg       0.26      0.50      0.34       329
weighted avg       0.28      0.53      0.36       329



In [ ]:
print("The mean accuracy score is", np.mean(CV_accuracy_array))

The mean accuracy score is 0.5565073224647692


In [ ]:
print("The standard deviation for accuracy is", np.std(CV_accuracy_array))

The standard deviation for accuracy is 0.019723091383869008


In [ ]:
print("The mean macro avg score is", np.mean(CV_macro_avg_array))

The mean macro avg score is 0.35743246561285746


In [ ]:
print("The standard deviation for macro avg score is", np.std(CV_macro_avg_array))

The standard deviation for macro avg score is 0.008167267905088729


In [ ]:
print("The mean weighted avg score is", np.mean(CV_weighted_avg_array))

The mean weighted avg score is 0.3981497137038236


In [ ]:
print("The standard deviation for weighted avg score is", np.std(CV_weighted_avg_array))

The standard deviation for weighted avg score is 0.023113605978325475


In [ ]:
from google.colab import runtime
runtime.unassign()
